<a href="https://colab.research.google.com/github/SanSan987/goit-de-hw-06/blob/main/dz6_den_chubar_oo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Встановлюємо необхідні пакети
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark pyspark==3.5.0 kafka-python

# Конфігурація середовища
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

import findspark
findspark.init()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.5/274.5 kB 15.1 MB/s eta 0:00:00


In [ ]:
# Завантаження Kafka
!wget https://archive.apache.org/dist/kafka/3.6.0/kafka_2.13-3.6.0.tgz
!tar -xzf kafka_2.13-3.6.0.tgz

# Запуск Zookeeper і Kafka
os.system("nohup kafka_2.13-3.6.0/bin/zookeeper-server-start.sh kafka_2.13-3.6.0/config/zookeeper.properties &")
os.system("nohup kafka_2.13-3.6.0/bin/kafka-server-start.sh kafka_2.13-3.6.0/config/server.properties &")


--2025-03-21 22:32:24--  https://archive.apache.org/dist/kafka/3.6.0/kafka_2.13-3.6.0.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 113257079 (108M) [application/x-gzip]
Saving to: ‘kafka_2.13-3.6.0.tgz’

kafka_2.13-3.6.0.tg 100%[===================>] 108.01M  27.3MB/s    in 4.7s    

2025-03-21 22:32:29 (22.9 MB/s) - ‘kafka_2.13-3.6.0.tgz’ saved [113257079/113257079]



0

In [ ]:
# Встановлення бібліотеки Kafka Python
!pip install kafka-python


In [ ]:
# Створення топіків
!kafka_2.13-3.6.0/bin/kafka-topics.sh --create --topic chub_building_sensors --bootstrap-server localhost:9092
!kafka_2.13-3.6.0/bin/kafka-topics.sh --create --topic chub_alerts --bootstrap-server localhost:9092


Created topic chub_building_sensors.
Created topic chub_alerts.


In [ ]:
# Перевірка створених топіків
from kafka.admin import KafkaAdminClient
admin_client = KafkaAdminClient(bootstrap_servers=['localhost:9092'])
print([topic for topic in admin_client.list_topics() if "chub" in topic])


['chub_building_sensors', 'chub_alerts']


In [ ]:
# Завантаження параметрів алертів з CSV
import pandas as pd
# Створюємо CSV-файл з параметрами
params_csv = """id,humidity_min,humidity_max,temperature_min,temperature_max,code,message
1,0,40,-999,-999,101,"It's too dry"
2,60,100,-999,-999,102,"It's too wet"
3,-999,-999,-300,30,103,"It's too cold"
4,-999,-999,40,300,104,"It's too hot"
"""

# Збереження параметрів у CSV-файл
with open("alert_params.csv", "w") as file:
    file.write(params_csv)

# Завантаження параметрів
alerts_df = pd.read_csv("alert_params.csv")
alerts_df.head()


,id,humidity_min,humidity_max,temperature_min,temperature_max,code,message
0,1,0,40,-999,-999,101,It's too dry
1,2,60,100,-999,-999,102,It's too wet
2,3,-999,-999,-300,30,103,It's too cold
3,4,-999,-999,40,300,104,It's too hot


In [ ]:
# Spark сесія
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("IoT_Alerts") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")


In [ ]:
# Скрипт для відправки даних до топіків, датчик 1
from kafka import KafkaProducer
import json, random, time
from datetime import datetime

producer = KafkaProducer(
    bootstrap_servers='localhost:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

sensor_id = random.randint(1000, 9999)

for _ in range(30):
    data = {
        "sensor_id": sensor_id,
        "timestamp": datetime.now().isoformat(),
        "temperature": random.randint(25, 45),
        "humidity": random.randint(15, 85)
    }
    print(f"Sensor {sensor_id} data sent:", data)
    producer.send("chub_building_sensors", data)
    producer.flush()  # негайне відправлення
    time.sleep(2)



Sensor 2005 data sent: {'sensor_id': 2005, 'timestamp': '2025-03-21T22:43:15.967703', 'temperature': 37, 'humidity': 33}
Sensor 2005 data sent: {'sensor_id': 2005, 'timestamp': '2025-03-21T22:43:18.039131', 'temperature': 26, 'humidity': 55}
Sensor 2005 data sent: {'sensor_id': 2005, 'timestamp': '2025-03-21T22:43:20.044445', 'temperature': 44, 'humidity': 79}
Sensor 2005 data sent: {'sensor_id': 2005, 'timestamp': '2025-03-21T22:43:22.048240', 'temperature': 41, 'humidity': 20}
Sensor 2005 data sent: {'sensor_id': 2005, 'timestamp': '2025-03-21T22:43:24.052130', 'temperature': 33, 'humidity': 83}
Sensor 2005 data sent: {'sensor_id': 2005, 'timestamp': '2025-03-21T22:43:26.057245', 'temperature': 27, 'humidity': 34}
Sensor 2005 data sent: {'sensor_id': 2005, 'timestamp': '2025-03-21T22:43:28.061897', 'temperature': 37, 'humidity': 54}
Sensor 2005 data sent: {'sensor_id': 2005, 'timestamp': '2025-03-21T22:43:30.066548', 'temperature': 30, 'humidity': 15}
Sensor 2005 data sent: {'sensor_

In [ ]:
# Скрипт для відправки даних до топіків, датчик 2
from kafka import KafkaProducer
import json, random, time
from datetime import datetime

producer = KafkaProducer(
    bootstrap_servers='localhost:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

sensor_id = random.randint(1000, 9999)

for _ in range(30):
    data = {
        "sensor_id": sensor_id,
        "timestamp": datetime.now().isoformat(),
        "temperature": random.randint(25, 45),
        "humidity": random.randint(15, 85)
    }
    print(f"Sensor {sensor_id} data sent:", data)
    producer.send("chub_building_sensors", data)
    producer.flush()  # негайне відправлення
    time.sleep(2)



Sensor 7540 data sent: {'sensor_id': 7540, 'timestamp': '2025-03-21T22:44:16.202226', 'temperature': 25, 'humidity': 37}
Sensor 7540 data sent: {'sensor_id': 7540, 'timestamp': '2025-03-21T22:44:18.208760', 'temperature': 29, 'humidity': 44}
Sensor 7540 data sent: {'sensor_id': 7540, 'timestamp': '2025-03-21T22:44:20.213982', 'temperature': 43, 'humidity': 38}
Sensor 7540 data sent: {'sensor_id': 7540, 'timestamp': '2025-03-21T22:44:22.218593', 'temperature': 29, 'humidity': 34}
Sensor 7540 data sent: {'sensor_id': 7540, 'timestamp': '2025-03-21T22:44:24.223001', 'temperature': 39, 'humidity': 69}
Sensor 7540 data sent: {'sensor_id': 7540, 'timestamp': '2025-03-21T22:44:26.227627', 'temperature': 36, 'humidity': 42}
Sensor 7540 data sent: {'sensor_id': 7540, 'timestamp': '2025-03-21T22:44:28.232574', 'temperature': 44, 'humidity': 67}
Sensor 7540 data sent: {'sensor_id': 7540, 'timestamp': '2025-03-21T22:44:30.238239', 'temperature': 39, 'humidity': 48}
Sensor 7540 data sent: {'sensor_

In [ ]:
# Spark Structured Streaming (Sliding Window + Watermark)
from pyspark.sql.functions import from_json, col, window, avg, current_timestamp, to_json, struct
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType

schema = StructType([
    StructField("sensor_id", IntegerType()),
    StructField("timestamp", TimestampType()),
    StructField("temperature", IntegerType()),
    StructField("humidity", IntegerType())
])

sensor_df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "chub_building_sensors") \
    .option("startingOffsets", "latest") \
    .load()

sensor_values = sensor_df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data")) \
    .select("data.*")

agg_df = sensor_values \
    .withWatermark("timestamp", "10 seconds") \
    .groupBy(window(col("timestamp"), "1 minute", "30 seconds")) \
    .agg(avg("temperature").alias("t_avg"), avg("humidity").alias("h_avg"))

# query з логуванням
query = agg_df.writeStream.outputMode("update") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination(180)
query.stop()


In [ ]:
# Перевірка алертів та запис у Kafka
from pyspark.sql.functions import current_timestamp, struct, to_json

alerts_spark_df = spark.createDataFrame(alerts_df)

alerts_final_df = agg_df.crossJoin(alerts_spark_df).where(
    ((col("humidity_min") != -999) & (col("h_avg") < col("humidity_min"))) |
    ((col("humidity_max") != -999) & (col("h_avg") > col("humidity_max"))) |
    ((col("temperature_min") != -999) & (col("t_avg") < col("temperature_min"))) |
    ((col("temperature_max") != -999) & (col("t_avg") > col("temperature_max")))
).select(
    "window", "t_avg", "h_avg", "code", "message", current_timestamp().alias("timestamp")
)

# Stream до Kafka
query_kafka = alerts_final_df.select(to_json(struct("*")).alias("value")).writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("topic", "chub_alerts") \
    .option("checkpointLocation", "/tmp/checkpoint") \
    .start()

# Stream до консолі
query_console = alerts_final_df.writeStream.outputMode("update") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query_kafka.awaitTermination(180)
query_console.awaitTermination(180)

query_kafka.stop()
query_console.stop()


In [ ]:
# Перевірка отриманих алертів
from kafka import KafkaConsumer
import json

consumer = KafkaConsumer(
    'chub_alerts',
    bootstrap_servers=['localhost:9092'],
    value_deserializer=lambda m: json.loads(m.decode('utf-8')),
    auto_offset_reset='earliest',
    consumer_timeout_ms=30000  # 30 секунд очікування
)

for message in consumer:
    print("🔥 ALERT:", message.value)

consumer.close()


🔥 ALERT: {'window': {'start': '2025-03-21T22:44:00.000Z', 'end': '2025-03-21T22:45:00.000Z'}, 't_avg': 35.46666666666667, 'h_avg': 51.4, 'code': 101, 'message': "It's too dry", 'timestamp': '2025-03-21T22:50:41.563Z'}
🔥 ALERT: {'window': {'start': '2025-03-21T22:44:00.000Z', 'end': '2025-03-21T22:45:00.000Z'}, 't_avg': 35.46666666666667, 'h_avg': 51.4, 'code': 103, 'message': "It's too cold", 'timestamp': '2025-03-21T22:50:41.563Z'}
🔥 ALERT: {'window': {'start': '2025-03-21T22:43:00.000Z', 'end': '2025-03-21T22:44:00.000Z'}, 't_avg': 34.0, 'h_avg': 51.86363636363637, 'code': 103, 'message': "It's too cold", 'timestamp': '2025-03-21T22:50:41.563Z'}
🔥 ALERT: {'window': {'start': '2025-03-21T22:43:30.000Z', 'end': '2025-03-21T22:44:30.000Z'}, 't_avg': 34.13333333333333, 'h_avg': 50.166666666666664, 'code': 103, 'message': "It's too cold", 'timestamp': '2025-03-21T22:50:41.563Z'}
🔥 ALERT: {'window': {'start': '2025-03-21T22:42:30.000Z', 'end': '2025-03-21T22:43:30.000Z'}, 't_avg': 35.0, 'h